In [1]:
import cv2
import numpy as np

def draw_annotations(image_path, annotations_path, output_path):
    # Load the image
    image = cv2.imread(image_path)

    # Read annotations
    with open(annotations_path, 'r') as file:
        lines = file.readlines()

    # Parse and draw annotations
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center, y_center, width, height = map(float, parts[1:])

        # Convert from relative coordinates to absolute coordinates
        img_height, img_width, _ = image.shape
        x_center *= img_width
        y_center *= img_height
        width *= img_width
        height *= img_height

        # Calculate the top-left and bottom-right coordinates of the bounding box
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        # Draw the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Draw the class label
        label = f"Class {class_id}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the output image
    cv2.imwrite(output_path, image)

# Example usage
image_path = '/home/achintya-trn0175/Desktop/24_03_05_130126-day-benchmark-f11-elp-3mm-y81i-75_0.png'
annotations_path = '/home/achintya-trn0175/Desktop/24_03_05_130126-day-benchmark-f11-elp-3mm-y81i-75_0.txt'
output_path = 'path_to_output_image.jpg'
draw_annotations(image_path, annotations_path, output_path)


In [ ]:
import cv2
import os
from ultralytics import YOLOv10

def load_yolo_model(model_path):
    # Load the pretrained YOLO model
    model = YOLOv10(model_path)
    return model

def detect_traffic_lights(model, image_path, class_name='traffic light'):
    # Load the image
    image = cv2.imread(image_path)
    # Perform inference
    results = model(image)
    # Extract traffic light annotations
    traffic_light_annotations = []
    for *box, conf, cls in results.xyxy[0]:
        if model.names[int(cls)] == class_name:
            x1, y1, x2, y2 = map(int, box)
            # Convert to YOLO format (normalized center coordinates and dimensions)
            img_height, img_width, _ = image.shape
            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height
            traffic_light_annotations.append((int(cls), x_center, y_center, width, height))
    return traffic_light_annotations

def update_annotations(annotations_path, new_annotations):
    # Read existing annotations
    with open(annotations_path, 'r') as file:
        lines = file.readlines()

    # Append new annotations
    with open(annotations_path, 'w') as file:
        for line in lines:
            file.write(line)
        for annotation in new_annotations:
            file.write(f"{annotation[0]} {annotation[1]:.6f} {annotation[2]:.6f} {annotation[3]:.6f} {annotation[4]:.6f}\n")

# Example usage
model_path='yolov10x.pt'
image_path = '/home/achintya-trn0175/Desktop/24_03_05_130126-day-benchmark-f11-elp-3mm-y81i-75_0.png'
annotations_path = 'path_to_your_annotations.txt'

# Load the model
model = load_yolo_model(model_path)

# Detect traffic lights
traffic_light_annotations = detect_traffic_lights(model, image_path)

# Update annotations file
update_annotations(annotations_path, traffic_light_annotations)
